In [1]:
# !pip install hyperopt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb


from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report
from sklearn.metrics import precision_recall_curve, confusion_matrix, mean_squared_error

from hyperopt import fmin, tpe, hp, partial

In [3]:
def f_score(y_true, y_pred, b=1):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    fscore = (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
    r_a_score = roc_auc_score(y_true, y_pred)
    np.nan_to_num(fscore, copy=False)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], r_a_score

In [4]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [5]:
df = pd.read_csv('churn_data.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

In [7]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Разобьем датасет на 3 выборки: трейн, тест и валидационную

In [9]:
x, X_test, y, y_test = train_test_split(df.drop('Exited', axis=1), df['Exited'], test_size=0.2, train_size=0.8)
X_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size = 0.25, train_size =0.75)

In [10]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [11]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler()),
            ])
    final_transformers.append((cont_col, cont_transformer))

In [12]:
feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

Создадим список пайплайнов с моделями

In [13]:
model_list = [LogisticRegression, 
              RandomForestClassifier, 
              GradientBoostingClassifier, 
              LGBMClassifier,
              XGBClassifier]
classifier_list = [make_pipeline(feats, model(random_state = 42)) for model in model_list]

Получим предикты от каждой из них и сведем все в датафрейм

In [14]:
names = []
metrics = []
for pipe in classifier_list:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict_proba(X_test)[:, 1]
    names.append(pipe.steps[1][0])
    thr, f1, pr, rec, roc = f_score(y_test.values, y_pred, b=1)
    tn, fp, fn, tp = confusion_matrix(y_test.values, y_pred > thr).ravel()
    metrics.append([thr, f1, pr, rec, roc, tp, tn, fp, fn, fp / (fp + tn)])

[18:22:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [15]:
m_cols = ['Threshhold', 'F-score', 'Precision', 'Recall', 'ROC-AUC', 'TP', 'TN', 'FP', 'FN', 'FPR']
result_df = pd.DataFrame(metrics, 
                         index=names, 
                         columns=m_cols)
result_df['FPR'] = result_df['FP'] / (result_df['FP'] + result_df['TN'])

In [16]:
result_df

,Threshhold,F-score,Precision,Recall,ROC-AUC,TP,TN,FP,FN,FPR
logisticregression,0.307705,0.488208,0.469388,0.508600,0.750519,206,1359,234,201,0.146893
randomforestclassifier,0.410000,0.600277,0.686709,0.533170,0.834033,211,1499,94,196,0.059008
gradientboostingclassifier,0.359124,0.611549,0.656338,0.572482,0.851091,232,1471,122,175,0.076585
lgbmclassifier,0.367175,0.621554,0.634271,0.609337,0.851255,247,1450,143,160,0.089768
xgbclassifier,0.316108,0.600715,0.583333,0.619165,0.835745,251,1413,180,156,0.112994


В принципе, все бустинговые модели отработали очень похоже, выберем для теста оптимизации параметров <b>lightgbm</b>

Оценим работу нашей модели на отложенной выборке

In [17]:
valid_pred = classifier_list[3].predict_proba(x_cv)[:, 1]

In [18]:
thr, f1, pr, rec, roc = f_score(y_cv.values, valid_pred, b=1)
tn, fp, fn, tp = confusion_matrix(y_cv.values, valid_pred > thr).ravel()
fpr = fp / (fp + tn)
n_met = [thr, f1, pr, rec, roc, tp, tn, fp, fn, fpr]

In [19]:
pd.Series({i: j for i, j in zip(m_cols, n_met)}, name='lgbm(on holdout)')

Threshhold       0.416449
F-score          0.625698
Precision        0.704403
Recall           0.562814
ROC-AUC          0.846188
TP             223.000000
TN            1508.000000
FP              94.000000
FN             175.000000
FPR              0.058677
Name: lgbm(on holdout), dtype: float64

Посчитаем следующим образом:

Деньги мы тратим на удержание клиентов, которые, как мы считаем, уходят. То есть это TN и FN. Но с FN мы получаем деньги, так как они на самом деле не уходят.  

На TP мы ничего не тратим, они приносят нам деньги  
На FP мы также ничего не тратим, так как считаем что они не уходят - деньги они не приносят  
На TN мы тратим деньги на удержание  

In [20]:
hold = -1
true_p = 2

tp * true_p + fp * 0 + (fn * hold + fn * true_p) + tn * hold

-887

Модель не целесообразна экономически

Попробуем оптимизировать.  

Код, с небольшими модификациями, <a href='https://www.programmersought.com/article/81271625281/'>отсюда</a>

In [21]:
x_tr, x_te, y_tr, y_te = train_test_split(feature_processing.fit_transform(df.drop(['Exited'], 
                                                                                   axis=1)), 
                                          df['Exited'], test_size=0.3, random_state=0)

In [22]:
train_data = lgb.Dataset(data=x_tr,label=y_tr)
test_data = lgb.Dataset(data=x_te,label=y_te)

In [23]:
# Customize the parameter space of hyperopt
space = {"max_depth": hp.randint("max_depth", 15),
         "num_trees": hp.randint("num_trees", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "bagging_fraction": hp.randint("bagging_fraction", 5),
         "num_leaves": hp.randint("num_leaves", 6),
         }
 
def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['num_trees'] = argsDict['num_trees'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["bagging_fraction"] = argsDict["bagging_fraction"] * 0.1 + 0.5
    argsDict["num_leaves"] = argsDict["num_leaves"] * 3 + 10
    if isPrint:
        print(argsDict)
    else:
        pass
 
    return argsDict

In [24]:
def lightgbm_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
 
    params = {'nthread': -1, # number of processes
                     'max_depth': argsDict['max_depth'], # maximum depth
                     'num_trees': argsDict['num_trees'], # number of trees
                     'eta': argsDict['learning_rate'], # learning rate
                     'bagging_fraction': argsDict['bagging_fraction'], # baggingsamples
                     'num_leaves': argsDict['num_leaves'], # The sum of the smallest sample of the endpoints
                     'objective': 'regression',
                     'feature_fraction': 0.7, #sample column sampling
                     'lambda_l1': 0, # L1 regularization
                     'lambda_l2': 0, # L2 Regularization
                     'bagging_seed': 100, # random seed, default in light 100
              }
    #rmse
    params['metric'] = ['rmse']
 
    model_lgb = lgb.train(params, train_data, num_boost_round=300, valid_sets=[test_data],early_stopping_rounds=100)
 
    return get_tranformer_score(model_lgb)
 
def get_tranformer_score(tranformer):
 
    model = tranformer
    prediction = model.predict(x_te, num_iteration=model.best_iteration)
 
    return mean_squared_error(y_te, prediction)

In [25]:
#Start using hyperopt for automatic parameter adjustment
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(lightgbm_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000377 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 874                      
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 25
[LightGBM] [Info] Start training from score 0.202286  
[1]	valid_0's rmse: 0.400007                          
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 0.395327                          
[3]	valid_0's rmse: 0.391478                          
[4]	valid_0's rmse: 0.38577                           
[5]	valid_0's rmse: 0.379858                          
[6]	valid_0's rmse: 0.376827                          
[7]	valid_0's rmse: 0.373153                          
[8]	valid_0's rmse: 0.368273                          
[9]	valid_0's rmse: 0.365268                          
[10]	valid_0's rmse: 0.361142                         
[11]	valid_0's rmse: 

[142]	valid_0's rmse: 0.314446                        
[143]	valid_0's rmse: 0.314459                        
[144]	valid_0's rmse: 0.314484                        
[145]	valid_0's rmse: 0.314529                        
[146]	valid_0's rmse: 0.314563                        
[147]	valid_0's rmse: 0.314561                        
[148]	valid_0's rmse: 0.314563                        
[149]	valid_0's rmse: 0.314583                        
[150]	valid_0's rmse: 0.314618                        
[151]	valid_0's rmse: 0.314603                        
[152]	valid_0's rmse: 0.314579                        
[153]	valid_0's rmse: 0.314574                        
[154]	valid_0's rmse: 0.314533                        
[155]	valid_0's rmse: 0.314524                        
[156]	valid_0's rmse: 0.314518                        
[157]	valid_0's rmse: 0.314448                        
[158]	valid_0's rmse: 0.314433                        
[159]	valid_0's rmse: 0.314443                        
[160]	vali

[58]	valid_0's rmse: 0.316949                                                    
[59]	valid_0's rmse: 0.316749                                                    
[60]	valid_0's rmse: 0.316591                                                    
[61]	valid_0's rmse: 0.316523                                                    
[62]	valid_0's rmse: 0.316424                                                    
[63]	valid_0's rmse: 0.316357                                                    
[64]	valid_0's rmse: 0.316321                                                    
[65]	valid_0's rmse: 0.316248                                                    
[66]	valid_0's rmse: 0.316171                                                    
[67]	valid_0's rmse: 0.316139                                                    
[68]	valid_0's rmse: 0.316064                                                    
[69]	valid_0's rmse: 0.315992                                                    
[70]	valid_0's r

[157]	valid_0's rmse: 0.315001                                                   
[158]	valid_0's rmse: 0.31515                                                    
[159]	valid_0's rmse: 0.315185                                                   
[160]	valid_0's rmse: 0.315169                                                   
[161]	valid_0's rmse: 0.315176                                                   
[162]	valid_0's rmse: 0.315188                                                   
[163]	valid_0's rmse: 0.315201                                                   
[164]	valid_0's rmse: 0.31523                                                    
[165]	valid_0's rmse: 0.315246                                                   
[166]	valid_0's rmse: 0.315255                                                   
[167]	valid_0's rmse: 0.315263                                                   
[168]	valid_0's rmse: 0.315347                                                   
[169]	valid_0's 

[54]	valid_0's rmse: 0.316146                                                    
[55]	valid_0's rmse: 0.316037                                                    
[56]	valid_0's rmse: 0.315944                                                    
[57]	valid_0's rmse: 0.315913                                                    
[58]	valid_0's rmse: 0.315892                                                    
[59]	valid_0's rmse: 0.315792                                                    
[60]	valid_0's rmse: 0.315774                                                    
[61]	valid_0's rmse: 0.315686                                                    
[62]	valid_0's rmse: 0.315614                                                    
[63]	valid_0's rmse: 0.315587                                                    
[64]	valid_0's rmse: 0.31555                                                     
[65]	valid_0's rmse: 0.315548                                                    
[66]	valid_0's r

...

[100]	valid_0's rmse: 0.314749                                                    
[101]	valid_0's rmse: 0.314776                                                    
[102]	valid_0's rmse: 0.314745                                                    
[103]	valid_0's rmse: 0.31477                                                     
[104]	valid_0's rmse: 0.314763                                                    
[105]	valid_0's rmse: 0.314768                                                    
[106]	valid_0's rmse: 0.314724                                                    
[107]	valid_0's rmse: 0.314707                                                    
[108]	valid_0's rmse: 0.314757                                                    
[109]	valid_0's rmse: 0.314791                                                    
[110]	valid_0's rmse: 0.314827                                                    
[111]	valid_0's rmse: 0.31481                                                     
[112

[198]	valid_0's rmse: 0.31497                                                     
[199]	valid_0's rmse: 0.314974                                                    
[200]	valid_0's rmse: 0.315006                                                    
[201]	valid_0's rmse: 0.314994                                                    
[202]	valid_0's rmse: 0.315018                                                    
[203]	valid_0's rmse: 0.314992                                                    
[204]	valid_0's rmse: 0.31494                                                     
[205]	valid_0's rmse: 0.314958                                                    
[206]	valid_0's rmse: 0.314955                                                    
[207]	valid_0's rmse: 0.314974                                                    
[208]	valid_0's rmse: 0.31499                                                     
[209]	valid_0's rmse: 0.31502                                                     
[210

[26]	valid_0's rmse: 0.32854                                                      
[27]	valid_0's rmse: 0.328203                                                     
[28]	valid_0's rmse: 0.327147                                                     
[29]	valid_0's rmse: 0.326134                                                     
[30]	valid_0's rmse: 0.325398                                                     
[31]	valid_0's rmse: 0.324442                                                     
[32]	valid_0's rmse: 0.323852                                                     
[33]	valid_0's rmse: 0.32332                                                      
[34]	valid_0's rmse: 0.322983                                                     
[35]	valid_0's rmse: 0.322356                                                     
[36]	valid_0's rmse: 0.321789                                                     
[37]	valid_0's rmse: 0.321441                                                     
[38]

[124]	valid_0's rmse: 0.314489                                                    
[125]	valid_0's rmse: 0.31445                                                     
[126]	valid_0's rmse: 0.314401                                                    
[127]	valid_0's rmse: 0.314398                                                    
[128]	valid_0's rmse: 0.314377                                                    
[129]	valid_0's rmse: 0.314318                                                    
[130]	valid_0's rmse: 0.314335                                                    
[131]	valid_0's rmse: 0.314292                                                    
[132]	valid_0's rmse: 0.31433                                                     
[133]	valid_0's rmse: 0.314349                                                    
[134]	valid_0's rmse: 0.314373                                                    
[135]	valid_0's rmse: 0.314355                                                    
[136

[222]	valid_0's rmse: 0.314546                                                    
[223]	valid_0's rmse: 0.314505                                                    
[224]	valid_0's rmse: 0.314492                                                    
[225]	valid_0's rmse: 0.314499                                                    
[226]	valid_0's rmse: 0.314516                                                    
[227]	valid_0's rmse: 0.314509                                                    
[228]	valid_0's rmse: 0.314508                                                    
[229]	valid_0's rmse: 0.314566                                                    
[230]	valid_0's rmse: 0.314566                                                    
[231]	valid_0's rmse: 0.314547                                                    
Early stopping, best iteration is:                                                
[131]	valid_0's rmse: 0.314292
100%|██████████| 20/20 [01:18<00:00,  3.94s/trial, best 

In [26]:
RMSE = lightgbm_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best lightgbm:', np.sqrt(RMSE))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 874
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 25
[LightGBM] [Info] Start training from score 0.202286
[1]	valid_0's rmse: 0.399284
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 0.393957
[3]	valid_0's rmse: 0.389614
[4]	valid_0's rmse: 0.383487
[5]	valid_0's rmse: 0.377345
[6]	valid_0's rmse: 0.37403
[7]	valid_0's rmse: 0.370073
[8]	valid_0's rmse: 0.365062
[9]	valid_0's rmse: 0.361843
[10]	valid_0's rmse: 0.357601
[11]	valid_0's rmse: 0.354844
[12]	valid_0's rmse: 0.35136
[13]	valid_0's rmse: 0.349163
[14]	valid_0's rmse: 0.34621
[15]	valid_0's rmse: 0.344128
[16]	valid_0's rmse: 0.34161
[17]	valid_0's rmse: 0.339826
[18]	valid_0's rmse: 0.339195
[19]	valid_0's rmse: 0.337617
[20]	valid_0's rmse: 0.336487
[21]	val

In [27]:
mod = LGBMClassifier(**best)

In [28]:
pip1 = make_pipeline(feats, mod)

In [29]:
pip1.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] num_iterations is set=402, num_trees=402 will be ignored. Current value: num_iterations=402


Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
       

In [30]:
y_pred = pip1.predict_proba(X_test)[:, 1]

thr, f1, pr, rec, roc = f_score(y_test.values, y_pred, b=1)
tn, fp, fn, tp = confusion_matrix(y_test.values, y_pred > thr).ravel()
fpr = fp / (fp + tn)

n_met = [thr, f1, pr, rec, roc, tp, tn, fp, fn, fpr]

In [31]:
result_df.append(pd.Series({i: j for i, j in zip(m_cols, n_met)}, name='lgbm(optimized)'))

,Threshhold,F-score,Precision,Recall,ROC-AUC,TP,TN,FP,FN,FPR
logisticregression,0.307705,0.488208,0.469388,0.508600,0.750519,206.0,1359.0,234.0,201.0,0.146893
randomforestclassifier,0.410000,0.600277,0.686709,0.533170,0.834033,211.0,1499.0,94.0,196.0,0.059008
gradientboostingclassifier,0.359124,0.611549,0.656338,0.572482,0.851091,232.0,1471.0,122.0,175.0,0.076585
lgbmclassifier,0.367175,0.621554,0.634271,0.609337,0.851255,247.0,1450.0,143.0,160.0,0.089768
xgbclassifier,0.316108,0.600715,0.583333,0.619165,0.835745,251.0,1413.0,180.0,156.0,0.112994
lgbm(optimized),0.225072,0.609566,0.556911,0.673219,0.845127,273.0,1375.0,218.0,134.0,0.136849


Оценим работу оптимизированной модели на отложенной выборке

In [32]:
valid_pred = pip1.predict_proba(x_cv)[:, 1]

In [33]:
thr, f1, pr, rec, roc = f_score(y_cv.values, valid_pred, b=1)
tn, fp, fn, tp = confusion_matrix(y_cv.values, valid_pred > thr).ravel()
fpr = fp / (fp + tn)
n_met = [thr, f1, pr, rec, roc, tp, tn, fp, fn, fpr]

In [34]:
pd.Series({i: j for i, j in zip(m_cols, n_met)}, name='lgbm(optimized on holdout)')

Threshhold       0.263558
F-score          0.603865
Precision        0.581395
Recall           0.628141
ROC-AUC          0.838297
TP             249.000000
TN            1422.000000
FP             180.000000
FN             149.000000
FPR              0.112360
Name: lgbm(optimized on holdout), dtype: float64

Можно сделать вывод что наш вариант оптимизации особой пользы не принес

Посчитаем экономику

In [35]:
hold = -1
true_p = 2

tp * true_p + fp * 0 + (fn * hold + fn * true_p) + tn * hold

-775

Модель не целесообразна экономически (хотя незначительное улучшение мы все таки получили)